### Bibliotecas

In [6]:
import os
import pandas as pd
import geopandas as gpd

### Diretórios

In [2]:
# DEFINIÇÕES DE DIRETÓRIOS
BASE_DIR = os.getcwd()
POLYGONS = os.path.join(BASE_DIR, "polygons")
DIR_ESTADO_SP = os.path.join(POLYGONS, "SP_UF_2022\\SP_UF_2022.shp")
DIR_MUNICIPIO_SP = os.path.join(POLYGONS, "limite_municipal\\LimiteMunicipal.shp")
DIR_DISTRITOS_BR = os.path.join(POLYGONS, "BR_Malha_Preliminar_Distrito_2022\\BR_Malha_Preliminar_Distrito_2022.gpkg")

### Funções Auxiliares

In [166]:
def calcular_centroide(file_gpd):
    """Calcula o centróide das geometrias e retorna em WKT."""
    # Reprojetando as geometrias para um CRS projetado (UTM Zone 23S para São Paulo)
    cidade_sp_proj = file_gpd.to_crs(epsg=31983)
    
    # Calculando o centróide no sistema de coordenadas projetadas
    cidade_sp_proj['centroide_proj'] = cidade_sp_proj.geometry.centroid
    
    # Reprojetando os centroides de volta para latitude e longitude
    file_gpd['CORD_CENTRAL'] = cidade_sp_proj['centroide_proj'].to_crs(epsg=4326).to_wkt()
    
    return file_gpd

In [17]:
def salvar_dados(file_gpd, output_filepath):
    """Salva o GeoDataFrame em um arquivo GeoJSON."""
    file_gpd = file_gpd.reset_index(drop=True)
    file_gpd.to_file(output_filepath, driver="GeoJSON")

### Extração Coordenadas

#### Estado de SP

In [168]:
estado_sp = gpd.read_file(DIR_ESTADO_SP)
estado_sp.head(3)

,CD_UF,NM_UF,SIGLA_UF,NM_REGIAO,AREA_KM2,geometry
0,35,São Paulo,SP,Sudeste\n,248219.485,"MULTIPOLYGON (((-46.47312 -22.70498, -46.47289..."


In [169]:
# Calcular centroide das coordenadas
estado_sp = calcular_centroide(estado_sp)
estado_sp

,CD_UF,NM_UF,SIGLA_UF,NM_REGIAO,AREA_KM2,geometry,CORD_CENTRAL
0,35,São Paulo,SP,Sudeste\n,248219.485,"MULTIPOLYGON (((-46.47312 -22.70498, -46.47289...",POINT (-48.747026 -22.262308)


In [170]:
# Salvando em GeoJSON
salvar_dados(estado_sp, "estado_sp.geojson")

#### Cidade de SP

In [171]:
# Visualizando dados
cidades_br = gpd.read_file(DIR_MUNICIPIO_SP)
cidades_br.head(3)

,Cod_ibge,Municipio,GID_RA,GID_RG,RA,RG,Area_Km2,GID_RM,GID_AU,RM,AU,geometry
0,3509908,Cananéia,16,30,RA de Registro,RG de Registro,1237.357,NaN,NaN,None,None,"POLYGON ((-47.83033 -24.75638, -47.83013 -24.7..."
1,3520426,Ilha Comprida,16,30,RA de Registro,RG de Registro,196.567,NaN,NaN,None,None,"POLYGON ((-47.4366 -24.67434, -47.43638 -24.67..."
2,3505401,Barra do Turvo,16,30,RA de Registro,RG de Registro,1007.670,NaN,NaN,None,None,"POLYGON ((-48.44457 -24.65134, -48.44418 -24.6..."


In [172]:
# Selecionando apenas a cidade de sp
cidade_sp = cidades_br[cidades_br['Municipio']=='São Paulo']

In [173]:
# Dropando colunas e padronizando nomes
cidade_sp = cidade_sp.drop(columns = {"GID_RA", "GID_RG", "RA", "RG", "GID_AU", "GID_RM","RM", "AU"})
cidade_sp = cidade_sp.rename(columns={"Cod_ibge": "CD_MUN", "Municipio":"NM_MUN", "Area_Km2": "AREA_KM2"})

In [174]:
# Calcular centroide das coordenadas
cidade_sp = calcular_centroide(cidade_sp)
cidade_sp

,CD_MUN,NM_MUN,AREA_KM2,geometry,CORD_CENTRAL
92,3550308,São Paulo,1521.11,"POLYGON ((-46.55066 -23.35714, -46.55011 -23.3...",POINT (-46.647972 -23.649892)


In [175]:
# Salvando em GeoJSON
salvar_dados(cidade_sp, "cidade_sp.geojson")

#### Distritos - Cidade SP

In [176]:
distrito_br = gpd.read_file(DIR_DISTRITOS_BR)
distrito_br.head(3)

,CD_REGIAO,NM_REGIAO,CD_UF,NM_UF,CD_MUN,NM_MUN,CD_DIST,NM_DIST,geometry
0,4,Sul,43,Rio Grande do Sul,None,None,None,None,"POLYGON ((-52.0972 -32.16167, -52.09926 -32.15..."
1,4,Sul,43,Rio Grande do Sul,None,None,None,None,"POLYGON ((-52.62752 -32.15022, -52.62816 -32.1..."
2,1,Norte,11,Rondônia,1100015,Alta Floresta D'Oeste,110001505,Alta Floresta D'Oeste,"POLYGON ((-61.93075 -11.97349, -61.93234 -11.9..."


In [183]:
# Selecionando apenas os distritos de SP
distrito_sp = distrito_br[distrito_br['NM_MUN']=='São Paulo']

In [184]:
# Dropando colunas e padronizando nomes
distrito_sp = distrito_sp.drop(columns = {"CD_REGIAO","NM_REGIAO", "CD_UF", "NM_UF"})

In [186]:
# Calcular centroide das coordenadas
distrito_sp = calcular_centroide(distrito_sp)

In [ ]:
excel_zonas_dist =pd.read_csv('Tabela_de_Distritos_e_Zonas.csv')

In [ ]:
# Renomear a coluna "Distrito" em excel_zonas_dist para "NM_DIST" para que coincida com distrito_sp
excel_zonas_dist.rename(columns={'Distrito': 'NM_DIST'}, inplace=True)

In [ ]:
# Fazer o merge dos dois dataframes usando a coluna NM_DIST
distrito_sp = distrito_sp.merge(excel_zonas_dist, on='NM_DIST', how='left')

In [33]:
# Salvando em GeoJSON
salvar_dados(distrito_sp, "distrito_sp.geojson")

In [34]:
distrito_sp = gpd.read_file('distrito_sp.geojson')
distrito_sp

,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CORD_CENTRAL,NOME_ZONA,geometry
0,3550308,São Paulo,355030801,Água Rasa,POINT (-46.571849 -23.56688),Zona Leste 1,"POLYGON ((-46.57207 -23.57948, -46.57212 -23.5..."
1,3550308,São Paulo,355030802,Alto de Pinheiros,POINT (-46.711886 -23.547577),Centro Ampliado,"POLYGON ((-46.69581 -23.55327, -46.69604 -23.5..."
2,3550308,São Paulo,355030803,Anhanguera,POINT (-46.792944 -23.430469),Zona Norte,"POLYGON ((-46.76157 -23.43178, -46.76157 -23.4..."
3,3550308,São Paulo,355030804,Aricanduva,POINT (-46.515679 -23.573925),Zona Leste 1,"POLYGON ((-46.51779 -23.58627, -46.51786 -23.5..."
4,3550308,São Paulo,355030805,Artur Alvim,POINT (-46.485831 -23.540212),Zona Leste 1,"POLYGON ((-46.48574 -23.55441, -46.48574 -23.5..."
...,...,...,...,...,...,...,...
91,3550308,São Paulo,355030892,Vila Medeiros,POINT (-46.577504 -23.488749),Zona Norte,"POLYGON ((-46.57783 -23.49527, -46.57794 -23.4..."
92,3550308,São Paulo,355030893,Vila Prudente,POINT (-46.572696 -23.592558),Zona Leste 1,"POLYGON ((-46.5748 -23.60107, -46.575 -23.6009..."
93,3550308,São Paulo,355030894,Vila Sônia,POINT (-46.739183 -23.59938),Centro Ampliado,"POLYGON ((-46.72791 -23.61149, -46.72798 -23.6..."
94,3550308,São Paulo,355030895,São Domingos,POINT (-46.747181 -23.493431),Zona Norte,"POLYGON ((-46.7467 -23.5075, -46.74671 -23.507..."


#### Zonas - Cidade SP

In [19]:
zonas_sp = gpd.read_file('zonas_sp.geojson')
zonas_sp

,Zona,CORD_CENTRAL,CD_MUN,geometry
0,Centro Ampliado,POINT (-46.679579 -23.582388),3550308,"POLYGON ((-46.67421 -23.64222, -46.67437 -23.6..."
1,Zona Leste 1,POINT (-46.542663 -23.561831),3550308,"POLYGON ((-46.57536 -23.6008, -46.57552 -23.60..."
2,Zona Leste 2,POINT (-46.443353 -23.549433),3550308,"POLYGON ((-46.45643 -23.64132, -46.4565 -23.64..."
3,Zona Norte,POINT (-46.673093 -23.45335),3550308,"POLYGON ((-46.67805 -23.51367, -46.67863 -23.5..."
4,Zona Sul,POINT (-46.708599 -23.813758),3550308,"POLYGON ((-46.60925 -23.90462, -46.60876 -23.9..."
